# V4系列で，バージョンごとに最終的な評価値を出す．(区間５カ月は行わない予定)
なんとなくどの　パラメータを可変していけば何が変わるのかの感覚がつかめたので，その感覚を大事にしながら総当たりでパラメータを可変していく．

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling


## 関数

In [3]:
def gainth_analysis(score_df,test_type='final'):
    
    gainth_arr=[100,110,120,140,160]
    index_arr=[0,1,2,3,4]
    cols=["gain_th","num_com","num_model(row)"]
    gainth_df=pd.DataFrame(columns=cols)
    def make_series(target_df):
        num_com=len(target_df['target_com'].value_counts().index)
        num_model=len(target_df)
        return num_com,num_model
    for gainth,i in zip(gainth_arr,index_arr):
        append_s=pd.Series(index=cols,dtype='object')
        if (i==0):
            target_df=score_df[score_df["gain_{}".format(test_type)]<gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="100未満"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        elif (i==4):
            target_df=score_df[score_df["gain_{}".format(test_type)]>=gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="150以上"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        else:
            target_df=score_df[(score_df["gain_{}".format(test_type)]<gainth_arr[i])&(score_df["gain_{}".format(test_type)]>=gainth_arr[i-1])]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="{}以上_{}未満".format(gainth_arr[i-1],gainth_arr[i])
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        gainth_df=gainth_df.append(append_s,ignore_index=True)
    return gainth_df

## バージョンごとに1DF 出力する(10会場分,かつV4_2だけで作成する，)


In [8]:

plus_m_th=3
#versions=['V4_1','V4_2']#,'V4_3']
version='V4_2'
#modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode='model_score_'
#mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す
mode_type='recent'
#きめじゃないパラメータの範囲設定(ループの数に注意！！)
gain_ths=np.arange(80, 180,4)
gain_std_ths=np.arange(10, 250,15)
hit_per_ths=np.arange(0,20,2)
hit_std_ths=np.arange(0, 2.0, 0.4)
num_hit_ths=[3,5,8,10,13,15]
place_master=master.get_place_master()
place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
version='V4_2'#バージョン
time_sta = time.time()



#コード本文
#for version in versions:
#    for mode,mode_type in zip(modes,mode_types):
for place_name in tqdm(place_names[:10]):
    analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
    print('version:{}_______mode:{}==================================================='.format(version,mode_type))
    #こっから下はバージョン，モードごと
    for num_hit_th in num_hit_ths:
        for gain_th in gain_ths:
            for gain_std_th in gain_std_ths:
                for hit_per_th in hit_per_ths:
                    for hit_std_th in hit_std_ths:
                        append_s=pd.Series(dtype='object')
                        for i in range(4):
                            now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                            dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
                            model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                            model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                            #決めのパラメータでいったん切り落とし
                            model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                            model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                            #可変していくパラメータ
                            target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                            target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                            target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                            target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                            analysis_df=gainth_analysis(target_df,test_type='final')
                            minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                            plus_df=analysis_df.loc['100以上_110未満':].copy()
                            try:
                                minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                            except ZeroDivisionError:
                                minus_model_per=100
                            append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
                            append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
                            append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
                        append_s['num_hit_th']=num_hit_th
                        append_s['gain_th']=gain_th
                        append_s['gain_std_th']=gain_std_th
                        append_s['hit_per_th']=hit_per_th
                        append_s['hit_std_th']=hit_std_th
                        analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
    analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
    analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
    analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
    csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th)
    analysis_v_df.to_csv(csv_path)

time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print('DONE')
print('elapsed time::{}=================================================================================================='.format(tim))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

version:V4_2_______mode:recent===================================================


 10%|███████▌                                                                   | 1/10 [5:31:47<49:46:10, 19907.81s/it]

version:V4_2_______mode:recent===================================================


 20%|██████████████▊                                                           | 2/10 [11:02:46<44:10:29, 19878.63s/it]

...

version:V4_2_______mode:recent===================================================


 90%|███████████████████████████████████████████████████████████████████▌       | 9/10 [51:38:43<5:49:02, 20942.86s/it]

version:V4_2_______mode:recent===================================================


100%|████████████████████████████████████████████████████████████████████████████| 10/10 [57:21:44<00:00, 20650.41s/it]

DONE
elapsed time::206504.06525063515==================================================================================================


## plus_monthも２カ月に減らした

In [9]:

plus_m_th=2
#versions=['V4_1','V4_2']#,'V4_3']
version='V4_2'
#modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode='model_score_'
#mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す
mode_type='recent'
#きめじゃないパラメータの範囲設定(ループの数に注意！！)
gain_ths=np.arange(80, 180,4)
gain_std_ths=np.arange(10, 250,15)
hit_per_ths=np.arange(0,20,2)
hit_std_ths=np.arange(0, 2.0, 0.4)
num_hit_ths=[3,5,8,10,13,15]
place_master=master.get_place_master()
place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
version='V4_2'#バージョン
time_sta = time.time()



#コード本文
#for version in versions:
#    for mode,mode_type in zip(modes,mode_types):
for place_name in tqdm(place_names[:10]):
    analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
    print('version:{}_______mode:{}==================================================='.format(version,mode_type))
    #こっから下はバージョン，モードごと
    for num_hit_th in num_hit_ths:
        for gain_th in gain_ths:
            for gain_std_th in gain_std_ths:
                for hit_per_th in hit_per_ths:
                    for hit_std_th in hit_std_ths:
                        append_s=pd.Series(dtype='object')
                        for i in range(4):
                            now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                            dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
                            model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                            model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                            #決めのパラメータでいったん切り落とし
                            model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                            model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                            #可変していくパラメータ
                            target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                            target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                            target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                            target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                            analysis_df=gainth_analysis(target_df,test_type='final')
                            minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                            plus_df=analysis_df.loc['100以上_110未満':].copy()
                            try:
                                minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                            except ZeroDivisionError:
                                minus_model_per=100
                            append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
                            append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
                            append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
                        append_s['num_hit_th']=num_hit_th
                        append_s['gain_th']=gain_th
                        append_s['gain_std_th']=gain_std_th
                        append_s['hit_per_th']=hit_per_th
                        append_s['hit_std_th']=hit_std_th
                        analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
    analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
    analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
    analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
    csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th)
    analysis_v_df.to_csv(csv_path)

time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print('DONE')
print('elapsed time::{}=================================================================================================='.format(tim))





  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

version:V4_2_______mode:recent===================================================


  0%|                                                                                         | 0/10 [2:29:56<?, ?it/s]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [3]:
# #きめのパラメータ
# place_name='asiya'
# plus_m_th=2
# num_hit_th=3
# versions=['V4_1','V4_2','V4_3']
# modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
# mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す
# #きめじゃないパラメータの範囲設定(ループの数に注意！！)
# gain_ths=np.arange(80, 180,4)
# gain_std_ths=np.arange(10, 250,15)
# hit_per_ths=np.arange(0,20,2)
# hit_std_ths=np.arange(0, 2.0, 0.4)
# time_sta = time.time()



# #コード本文
# for version in versions:
#     for mode,mode_type in zip(modes,mode_types):
#         analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
#         print('version:{}_______mode:{}==================================================='.format(version,mode_type))
#         #こっから下はバージョン，モードごと
#         for gain_th in tqdm(gain_ths):
#             for gain_std_th in gain_std_ths:
#                 for hit_per_th in hit_per_ths:
#                     for hit_std_th in hit_std_ths:
#                         append_s=pd.Series(dtype='object')
#                         for i in range(4):
#                             now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
#                             dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
#                             model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
#                             model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
#                             #決めのパラメータでいったん切り落とし
#                             model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
#                             model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
#                             #可変していくパラメータ
#                             target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
#                             target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
#                             target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
#                             target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
#                             analysis_df=gainth_analysis(target_df,test_type='final')
#                             minus_df=analysis_df.loc[:'100以上_110未満'].copy()
#                             plus_df=analysis_df.loc['100以上_110未満':].copy()
#                             try:
#                                 minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
#                             except ZeroDivisionError:
#                                 minus_model_per=100
#                             append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
#                             append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
#                             append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
#                         append_s['gain_th']=gain_th
#                         append_s['gain_std_th']=gain_std_th
#                         append_s['hit_per_th']=hit_per_th
#                         append_s['hit_std_th']=hit_std_th
#                         analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
#         analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
#         analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
#         analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
#         csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode_type}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version)
#         analysis_v_df.to_csv(csv_path)

# time_end = time.time()
# # 経過時間（秒）
# tim = time_end- time_sta

# print('DONE')
# print('elapsed time::{}=================================================================================================='.format(tim))





  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:period===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

...

version:V4_3_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_3_______mode:period===================================================


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [3:53:18<00:00, 559.96s/it]

DONE
elapsed time::39680.50764346123==================================================================================================


# 芦屋のみで動かし，機能部分を作ってみる

## きめのパラメータの変更

## hit も可変していく，num_plus_monthに関しては3

In [32]:
#きめのパラメータ
place_name='asiya'
plus_m_th=3
versions=['V4_1','V4_2']#,'V4_3']
modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す
#きめじゃないパラメータの範囲設定(ループの数に注意！！)
gain_ths=np.arange(80, 180,4)
gain_std_ths=np.arange(10, 250,15)
hit_per_ths=np.arange(0,20,2)
hit_std_ths=np.arange(0, 2.0, 0.4)
num_hit_ths=[3,5,8,10,13,15]
time_sta = time.time()



#コード本文
for version in versions:
    for mode,mode_type in zip(modes,mode_types):
        analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
        print('version:{}_______mode:{}==================================================='.format(version,mode_type))
        #こっから下はバージョン，モードごと
        for num_hit_th in num_hit_ths:
            for gain_th in tqdm(gain_ths):
                for gain_std_th in gain_std_ths:
                    for hit_per_th in hit_per_ths:
                        for hit_std_th in hit_std_ths:
                            append_s=pd.Series(dtype='object')
                            for i in range(4):
                                now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                                dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
                                model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                                model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                                #決めのパラメータでいったん切り落とし
                                model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                                model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                                #可変していくパラメータ
                                target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                                target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                                target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                                target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                                analysis_df=gainth_analysis(target_df,test_type='final')
                                minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                                plus_df=analysis_df.loc['100以上_110未満':].copy()
                                try:
                                    minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                                except ZeroDivisionError:
                                    minus_model_per=100
                                append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
                                append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
                                append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
                            append_s['num_hit_th']=num_hit_th
                            append_s['gain_th']=gain_th
                            append_s['gain_std_th']=gain_std_th
                            append_s['hit_per_th']=hit_per_th
                            append_s['hit_std_th']=hit_std_th
                            analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
        analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
        analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
        analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
        csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th)
        analysis_v_df.to_csv(csv_path)

time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print('DONE')
print('elapsed time::{}=================================================================================================='.format(tim))





  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:period===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_2_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_2_______mode:period===================================================


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:01:55<00:00, 148.62s/it]


DONE
elapsed time::86629.49615955353==================================================================================================


## plus_monthを２に減らした

In [33]:
#きめのパラメータ
place_name='asiya'
plus_m_th=2
versions=['V4_1','V4_2']#,'V4_3']
modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す
#きめじゃないパラメータの範囲設定(ループの数に注意！！)
gain_ths=np.arange(80, 180,4)
gain_std_ths=np.arange(10, 250,15)
hit_per_ths=np.arange(0,20,2)
hit_std_ths=np.arange(0, 2.0, 0.4)
num_hit_ths=[3,5,8,10,13,15]
time_sta = time.time()



#コード本文
for version in versions:
    for mode,mode_type in zip(modes,mode_types):
        analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
        print('version:{}_______mode:{}==================================================='.format(version,mode_type))
        #こっから下はバージョン，モードごと
        for num_hit_th in num_hit_ths:
            for gain_th in tqdm(gain_ths):
                for gain_std_th in gain_std_ths:
                    for hit_per_th in hit_per_ths:
                        for hit_std_th in hit_std_ths:
                            append_s=pd.Series(dtype='object')
                            for i in range(4):
                                now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                                dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
                                model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                                model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                                #決めのパラメータでいったん切り落とし
                                model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                                model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                                #可変していくパラメータ
                                target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                                target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                                target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                                target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                                analysis_df=gainth_analysis(target_df,test_type='final')
                                minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                                plus_df=analysis_df.loc['100以上_110未満':].copy()
                                try:
                                    minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                                except ZeroDivisionError:
                                    minus_model_per=100
                                append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
                                append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
                                append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
                            append_s['num_hit_th']=num_hit_th
                            append_s['gain_th']=gain_th
                            append_s['gain_std_th']=gain_std_th
                            append_s['hit_per_th']=hit_per_th
                            append_s['hit_std_th']=hit_std_th
                            analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
        analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
        analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
        analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
        csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th)
        analysis_v_df.to_csv(csv_path)

time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print('DONE')
print('elapsed time::{}=================================================================================================='.format(tim))





  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_1_______mode:period===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_2_______mode:recent===================================================


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

version:V4_2_______mode:period===================================================


 20%|████████████████▍                                                                 | 5/25 [13:02<52:09, 156.49s/it]


PermissionError: [Errno 13] Permission denied

target_com               0.283059
depth                   -0.005746
target_per              -0.186508
threshold               -0.111219
total_get_test           0.084398
total_use_test           0.226903
num_com_test             0.046510
num_pred_test            0.226903
gain_test               -0.257171
gain_std_test            0.172104
num_hit_test            -0.028523
buy_hit_per_test        -0.266918
buy_hit_per_std_test    -0.259625
plus_month_num_test      0.030133
diff_mea_med_test       -0.088409
total_get_final          0.833576
total_use_final          0.201764
num_com_final            0.200710
num_pred_final           0.201764
gain_final               1.000000
gain_std_final           0.595621
num_hit_final            0.361705
buy_hit_per_final        0.403838
buy_hit_per_std_final    0.097588
plus_month_num_final     0.849329
diff_mea_med_final       0.335706
Name: gain_final, dtype: float64

## 仮での機能テスト

In [58]:
# i=0
# version=versions[0]
# analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
# mode=modes[0]
# now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
# time_sta = time.time()
# #こっから下はバージョン，モードごと
# for gain_th in tqdm(gain_ths):
#     for gain_std_th in gain_std_ths:
#         for hit_per_th in hit_per_ths:
#             for hit_std_th in hit_std_ths:
#                 append_s=pd.Series(dtype='object')
#                 for i in range(4):
#                     now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
#                     dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
#                     model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
#                     model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
#                     #決めのパラメータでいったん切り落とし
#                     model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
#                     model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
#                     target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
#                     target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
#                     target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
#                     target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
#                     analysis_df=gainth_analysis(target_df,test_type='final')
#                     minus_df=analysis_df.loc[:'100以上_110未満'].copy()
#                     plus_df=analysis_df.loc['100以上_110未満':].copy()
#                     try:
#                         minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
#                     except ZeroDivisionError:
#                         minus_model_per=100
#                     append_s['st_{}_minus_per(model)'.format(now_ym)]=minus_model_per
#                     append_s['st_{}_num_plus_com'.format(now_ym)]=plus_df['num_com'].sum()
#                     append_s['st_{}_num_plus_model(row)'.format(now_ym)]=plus_df['num_model(row)'].sum()
#                 append_s['gain_th']=gain_th
#                 append_s['gain_std_th']=gain_std_th
#                 append_s['hit_per_th']=hit_per_th
#                 append_s['hit_std_th']=hit_std_th
#                 analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
            
# analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
# analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
# analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
# csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode_type}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version)

# analysis_v_df.to_csv(csv_path)

# time_end = time.time()
# # 経過時間（秒）
# tim = time_end- time_sta

# print('DONE')
# print('elapsed time::{}=================================================================================================='.format(tim))




  0%|                                                                                           | 0/25 [00:44<?, ?it/s]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [62]:
mode_type='test'
analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_20200101_minus_per(model)']+analysis_v_df['st_20200401_minus_per(model)']+analysis_v_df['st_20200701_minus_per(model)']+analysis_v_df['st_20201001_minus_per(model)'])/4)        
analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_20200101_num_plus_com']+analysis_v_df['st_20200401_num_plus_com']+analysis_v_df['st_20200701_num_plus_com']+analysis_v_df['st_20201001_num_plus_com'])/4)        
analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_20200101_num_plus_model(row)']+analysis_v_df['st_20200401_num_plus_model(row)']+analysis_v_df['st_20200701_num_plus_model(row)']+analysis_v_df['st_20201001_num_plus_model(row)'])/4)        
csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode_type}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version)
analysis_v_df.to_csv(csv_path)

In [61]:
version

'V4_1'